In [1]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import timeit
from datetime import datetime

In [2]:
# First method using tf.function() [eager vs graph]
x = tf.random.uniform(shape=[10,10],minval = -1,maxval = 2,dtype = tf.dtypes.int32)

def power(x,y):
  result = tf.eye(10,dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x,result)
  return result

In [3]:
print("Eager Execution: ",timeit.timeit(lambda : power(x,100),number = 1000),"seconds")

Eager Execution:  5.028842370999996 seconds


In [4]:
power_graph = tf.function(power)
print("Graph execution: ",timeit.timeit(lambda : power_graph(x,100),number=1000),"seconds")

Graph execution:  0.9904186790000153 seconds


In [5]:
# Second method using decorator
@tf.function
def add_num(a,b):
  return tf.add(a,b,name='sum')

a=tf.constant(5,name='a')
b=tf.constant(7,name='b')

result = add_num(a,b)
print("Result: ",result.numpy())
print("Graph execution: ",timeit.timeit(lambda : add_num(a,b),number=1000),"seconds")

Result:  12
Graph execution:  0.510528254999997 seconds


In [6]:
from keras.layers import Dense, Conv2D, BatchNormalization
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Input, Flatten,Dropout
from keras.layers import concatenate, Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.utils import plot_model, to_categorical

In [7]:
#Image Classification using DenseNet

cifar10 = keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 12s 0us/step


In [8]:
batch_size = 64
epochs = 8
data_augmentation = True

num_classes = 10
num_dense_blocks = 3
use_max_pool = False

In [9]:
growth_rate = 12
depth = 100
num_bottleneck_layers = (depth - 4) // (2*num_dense_blocks)

num_filters_dense_blocks = 2*growth_rate
compression_factor = 0.5

In [10]:
input_shape = x_train.shape[1:]

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

In [11]:
def lr_schedule(epoch):
  lr = 1e-3
  if epoch > 180:
    lr *= 0.5e-3
  elif epoch > 160:
    lr *= 1e-3
  elif epoch > 120:
    lr *= 1e-2
  elif epoch > 80:
    lr *= 1e-1
  print("Learning Rate: ",lr)
  return lr

In [12]:
inputs = Input(shape = input_shape)
x = BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(num_filters_dense_blocks,kernel_size = 3,padding = 'same',kernel_initializer='he_normal')(x)
x = concatenate([inputs,x])

In [13]:
for i in range(num_dense_blocks):
  for j in range(num_bottleneck_layers):
    y = BatchNormalization()(x)
    y = Activation('relu')(y)
    y = Conv2D(4*growth_rate,kernel_size=1,padding='same',kernel_initializer='he_normal')(y)

    if not data_augmentation:
      y = Dropout(0.2)(y)
      y = BatchNormalization()(y)
      y = Activation('relu')(y)
      y = Conv2D(growth_rate,kernel_size=3,padding='same',kernel_initializer='he_normal')(y)
    if not data_augmentation:
      y = Dropout(0.2)(y)
    x = concatenate([x,y])

  # no transition layer after last dense block
  if i == num_dense_blocks - 1:
    continue

  # transition layer compresses num of feature maps & reduces size by 2
  num_filters_dense_blocks += num_bottleneck_layers * growth_rate
  num_filters_dense_blocks = int(num_filters_dense_blocks * compression_factor)
  y = BatchNormalization()(x)
  y = Conv2D(num_filters_dense_blocks,kernel_size=1,padding='same',kernel_initializer='he_normal')(y)
  if not data_augmentation:
    y = Dropout(0.2)(y)
  x = AveragePooling2D()(y)

In [14]:
x = AveragePooling2D(pool_size=8)(x)
y = Flatten()(x)

outputs = Dense(num_classes, kernel_initializer='he_normal', activation='softmax')(y)

In [15]:
model = Model(inputs=inputs,outputs=outputs)

model.compile(loss='categorical_crossentropy',
              optimizer = RMSprop(1e-3),
              metrics = ['acc'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 3)            12        ['input_1[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 32, 32, 3)            0         ['batch_normalization[0][0]'] 
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 24)           672       ['activation[0][0]']      

In [16]:
save_dir = os.path.join(os.getcwd(),'saved_models')
model_name = 'cifar10_densenet_model.{epoch:02d}.h5'
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)
filepath = os.path.join(save_dir,model_name)

In [17]:
checkpoint = ModelCheckpoint(filepath=filepath, moniter='val_acc', verbose=1, save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5,min_lr=0.5e-6)

callbacks = [checkpoint,lr_reducer,lr_scheduler]

In [18]:
if not data_augmentation:
  print('Not using data augmentation.')
  model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test), shuffle=True, callbacks=callbacks)
else:
    print('Using real-time augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False,
        zca_whitening=False, rotation_range =0, width_shift_range=0.1, height_shift_range=0.1,
        horizontal_flip=True, vertical_flip=False)

    datagen.fit(x_train)
    steps_per_epoch = math.ceil(len(x_train)/ batch_size)

    model.fit(x=datagen.flow(x_train,y_train,batch_size=batch_size),
              verbose=1,
              epochs=epochs,
              validation_data = (x_test,y_test),
              steps_per_epoch=steps_per_epoch,
              callbacks=callbacks)

Using real-time augmentation.
Learning Rate:  0.001
Epoch 1/8
782/782 [==============================] - ETA: 0s - loss: 1.9248 - acc: 0.3291
Epoch 1: val_loss improved from inf to 1.74535, saving model to /content/saved_models/cifar10_densenet_model.01.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 297s 322ms/step - loss: 1.9248 - acc: 0.3291 - val_loss: 1.7454 - val_acc: 0.3599 - lr: 0.0010
Learning Rate:  0.001
Epoch 2/8
782/782 [==============================] - ETA: 0s - loss: 1.5232 - acc: 0.4512
Epoch 2: val_loss did not improve from 1.74535
782/782 [==============================] - 254s 325ms/step - loss: 1.5232 - acc: 0.4512 - val_loss: 3.0293 - val_acc: 0.3202 - lr: 0.0010
Learning Rate:  0.001
Epoch 3/8
782/782 [==============================] - ETA: 0s - loss: 1.3945 - acc: 0.4977
Epoch 3: val_loss did not improve from 1.74535
782/782 [==============================] - 254s 325ms/step - loss: 1.3945 - acc: 0.4977 - val_loss: 2.2022 - val_acc: 0.4061 - lr: 0.0010
Learning Rate:  0.001
Epoch 4/8
782/782 [==============================] - ETA: 0s - loss: 1.2956 - acc: 0.5339
Epoch 4: val_loss improved from 1.74535 to 1.46731, saving model to /content/saved_models/cifar10_densenet_model.04.h5
782/782 [============================

In [20]:
scores = model.evaluate(x_test,y_test,verbose=0)
print("Test loss: ", scores[0])
print("Test accuracy: ", scores[1])

Test loss:  1.4282664060592651
Test accuracy:  0.5462999939918518


The Accuracy is low because of lower trainable(less epochs) data.